In [1]:
import sys
import numpy as np
import tensorflow as tf
import argparse
from utils import splitter, normalize
from model import graph

In [2]:
trainpath = './fashionmnist/training2.csv'
validationpath = './fashionmnist/validation.csv'

In [3]:
default_values = [[0.0] for _ in range(785)]

In [4]:
def decode(line, default_values=default_values):
    item = tf.decode_csv(line, default_values)
    return item[0], item[1:]

In [5]:
training_dataset = tf.data.TextLineDataset(trainpath).skip(1).filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#"))
validation_dataset = tf.data.TextLineDataset(validationpath).skip(1).filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#"))

In [6]:
training_dataset = (training_dataset.cache().map(decode))

In [7]:
# training_dataset = training_dataset.map(lambda x: tf.py_func(splitter, [x], [tf.float32, tf.float32]))
# validation_dataset = validation_dataset.map(lambda x: tf.py_func(splitter, [x], [tf.float32, tf.float32]))

In [8]:
training_dataset = training_dataset.map(lambda label, pixel: tf.py_func(normalize, [label, pixel], [tf.float32, tf.float32]))

In [11]:
training_dataset = training_dataset.shuffle(buffer_size=10000)

In [9]:
training_dataset = training_dataset.batch(10)
validation_dataset = validation_dataset.batch(100)

In [10]:
# A feedable iterator is defined by a handle placeholder and its structure. We
# could use the `output_types` and `output_shapes` properties of either
# `training_dataset` or `validation_dataset` here, because they have
# identical structure.
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

In [11]:
training_dataset.output_shapes

(TensorShape(None), TensorShape(None))

In [12]:
# You can use feedable iterators with a variety of different kinds of iterator
# (such as one-shot and initializable iterators).
training_iterator = training_dataset.make_initializable_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()

In [13]:
with tf.Session() as sess:
    # The `Iterator.string_handle()` method returns a tensor that can be evaluated
    # and used to feed the `handle` placeholder.
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    
    for i in range(1):
        sess.run(training_iterator.initializer)
        while True:
            try:
                label_batch, image_batch = sess.run(next_element, feed_dict={handle: training_handle})
                print(label_batch)
            except tf.errors.OutOfRangeError:
                print('Out of Data, Training Finished!')
#             finally:
#                 sess.run(validation_iterator.initializer)
#                 sess.run(next_element, feed_dict={handle: validation_handle})
#         if i%10==0:
#             sess.run(validation_iterator.initializer)
#             sess.run(next_element, feed_dict={handle: validation_handle})

[ 2.  3.  7.  0.  1.  5.  9.  7.  8.  4.]
[ 5.  2.  8.  1.  9.  1.  6.  8.  5.  3.]
[ 9.  3.  6.  3.  9.  5.  7.  7.  5.  0.]
[ 0.  0.  9.  4.  9.  2.  2.  6.  5.  8.]
[ 1.  6.  4.  2.  2.  7.  9.  2.  1.  3.]
[ 4.  5.  9.  2.  3.  8.  8.  6.  5.  5.]
[ 4.  5.  6.  1.  4.  7.  2.  7.  1.  5.]
[ 6.  2.  5.  2.  3.  0.  8.  4.  3.  9.]
[ 9.  9.  8.  9.  3.  4.  3.  2.  0.  5.]
[ 7.  3.  7.  7.  0.  6.  1.  0.  1.  9.]
[ 0.  7.  3.  0.  4.  2.  8.  9.  3.  8.]
[ 6.  1.  0.  9.  9.  2.  0.  2.  3.  4.]
[ 2.  9.  7.  0.  1.  8.  8.  1.  7.  8.]
[ 2.  5.  6.  1.  3.  6.  6.  7.  0.  0.]
[ 1.  4.  5.  8.  5.  8.  3.  9.  6.  8.]
[ 6.  7.  7.  2.  9.  1.  5.  2.  2.  0.]
[ 6.  0.  3.  9.  4.  9.  0.  0.  0.  4.]
[ 5.  5.  1.  4.  0.  9.  0.  2.  5.  7.]
[ 2.  1.  9.  1.  5.  4.  5.  3.  9.  6.]
[ 6.  1.  3.  1.  6.  3.  4.  2.  6.  5.]
[ 4.  0.  2.  1.  8.  9.  7.  8.  4.  6.]
[ 5.  7.  1.  5.  1.  5.  6.  5.  7.  3.]
[ 8.  8.  3.  7.  1.  5.  5.  4.  6.  3.]
[ 5.  3.  6.  4.  7.  1.  0.  3.  

KeyboardInterrupt: 